In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C:/Users/tmost/Documents/Data-course/Salary/glassdoor_jobs.csv')
df.tail(3)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1
954,954,Data Engineer,-1,Loading...\n\nTitle: Data Engineer\n\nLocation...,4.8,IGNW\n4.8,"Austin, TX","Portland, OR",201 to 500 employees,2015,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),Slalom
955,955,Research Scientist – Security and Privacy,$61K-$126K (Glassdoor est.),Returning Candidate? Log back in to the Career...,3.6,Riverside Research Institute\n3.6,"Beavercreek, OH","Arlington, VA",501 to 1000 employees,1967,Nonprofit Organization,Federal Agencies,Government,$50 to $100 million (USD),-1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         956 non-null    int64  
 1   Job Title          956 non-null    object 
 2   Salary Estimate    956 non-null    object 
 3   Job Description    956 non-null    object 
 4   Rating             956 non-null    float64
 5   Company Name       956 non-null    object 
 6   Location           956 non-null    object 
 7   Headquarters       956 non-null    object 
 8   Size               956 non-null    object 
 9   Founded            956 non-null    int64  
 10  Type of ownership  956 non-null    object 
 11  Industry           956 non-null    object 
 12  Sector             956 non-null    object 
 13  Revenue            956 non-null    object 
 14  Competitors        956 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 112.2+ KB


In [4]:
df.isnull().any()

Unnamed: 0           False
Job Title            False
Salary Estimate      False
Job Description      False
Rating               False
Company Name         False
Location             False
Headquarters         False
Size                 False
Founded              False
Type of ownership    False
Industry             False
Sector               False
Revenue              False
Competitors          False
dtype: bool

# Cleaning

## Salary parsing

In [5]:
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [8]:
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

In [9]:
df = df[df['Salary Estimate'] != '-1'] # out in '' as it is object

In [10]:
df.shape

(742, 17)

In [14]:
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0].strip())

In [15]:
salary.head()

0     $53K-$91K
1    $63K-$112K
2     $80K-$90K
3     $56K-$97K
4    $86K-$143K
Name: Salary Estimate, dtype: object

In [16]:
minus_kd = salary.apply(lambda x: x.replace('K','').replace('$',''))

In [18]:
minus_kd.head()

0     53-91
1    63-112
2     80-90
3     56-97
4    86-143
Name: Salary Estimate, dtype: object

In [19]:
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary',''))

In [21]:
min_hr.head()

0     53-91
1    63-112
2     80-90
3     56-97
4    86-143
Name: Salary Estimate, dtype: object

In [22]:
#df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))

df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0].split(':')[1]) if ':' in x else int(x.split('-')[0]))

In [23]:
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))

In [24]:
df['avg_salary'] = (df.min_salary + df.max_salary)/2

In [25]:
## Company name text only

In [26]:
df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] < 0 else x['Company Name'][:-3],axis=1)# 3 last charachter

In [ ]:
## State Field 

In [30]:
df['Location']

0      Albuquerque, NM
1        Linthicum, MD
2       Clearwater, FL
3         Richland, WA
4         New York, NY
            ...       
950      Cambridge, MA
951      Nashville, TN
952     Pittsburgh, PA
953      Allentown, PA
955    Beavercreek, OH
Name: Location, Length: 742, dtype: object

In [32]:
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])

In [34]:
# check to see if the actuak job position is at the headquarters
df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0 , axis =1)

In [ ]:
##  Age of company

In [39]:
df['age'] = df.Founded.apply(lambda x: x if x <1 else 2023 - x)

In [41]:
df['age'] = df['Founded'].apply(lambda x: 2023 - x if x > 0 else None)

In [ ]:
## parsing of job decription (python,etc)

In [48]:
df['Job Description']

0      Data Scientist\nLocation: Albuquerque, NM\nEdu...
1      What You Will Do:\n\nI. General Summary\n\nThe...
2      KnowBe4, Inc. is a high growth information sec...
3      *Organization and Job ID**\nJob ID: 310709\n\n...
4      Data Scientist\nAffinity Solutions / Marketing...
                             ...                        
950    Site Name: USA - Massachusetts - Cambridge\nPo...
951    THE CHALLENGE\nEventbrite has a world-class da...
952    The Auton Lab at Carnegie Mellon University is...
953    Data Science ManagerResponsibilities:\n\nOvers...
955    Returning Candidate? Log back in to the Career...
Name: Job Description, Length: 742, dtype: object

-- check if it is exist in that position

- python
- r studio
- spark
- aws
- excel

In [49]:
df['python-yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [51]:
df['python-yn'].value_counts()

1    392
0    350
Name: python-yn, dtype: int64

In [52]:
df['R_yn'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)

In [53]:
df['R_yn'].value_counts()

0    740
1      2
Name: R_yn, dtype: int64

In [54]:
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)

In [55]:
df['spark'].value_counts()

0    575
1    167
Name: spark, dtype: int64

In [58]:
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [59]:
df['aws'].value_counts()

0    566
1    176
Name: aws, dtype: int64

In [60]:
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

In [61]:
df['excel'].value_counts()

1    388
0    354
Name: excel, dtype: int64

In [ ]:
## drop the fisrt column (the unnamed columns)

In [62]:
df = df.drop([df.columns[0]], axis='columns')

In [64]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,avg_salary,company_txt,job_state,same_state,age,python-yn,R_yn,spark,aws,excel
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,...,72.0,Tecolote Research\n,NM,0,50.0,1,0,0,0,1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,...,87.5,University of Maryland Medical System\n,MD,0,39.0,1,0,0,0,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,...,85.0,KnowBe4\n,FL,1,13.0,1,0,1,0,1
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,...,76.5,PNNL\n,WA,1,58.0,1,0,0,0,0
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,114.5,Affinity Solutions\n,NY,1,25.0,1,0,0,0,1


In [65]:
df.to_csv('salary_data_cleaned.csv',index=False)